In [7]:
!pip install plotly_express
!wget -O tracelib.py https://raw.githubusercontent.com/HiEST/bojos/master/traces/tracelib.py
#!wget -O emissions_eq_pyc.pyc https://raw.githubusercontent.com/HiEST/bojos/master/traces/emissions_eq_pyc.pyc
!wget -O emissions_eq.py https://raw.githubusercontent.com/HiEST/bojos/master/traces/emissions_eq.py

--2020-11-06 12:56:00--  https://raw.githubusercontent.com/HiEST/bojos/master/traces/tracelib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2536 (2.5K) [text/plain]
Saving to: ‘tracelib.py’

tracelib.py         100%[===================>]   2.48K  --.-KB/s    in 0s      

2020-11-06 12:56:00 (32.0 MB/s) - ‘tracelib.py’ saved [2536/2536]

--2020-11-06 12:56:00--  https://raw.githubusercontent.com/HiEST/bojos/master/traces/emissions_eq.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5366 (5.2K) [text/plain]
Saving to: ‘emissions_eq.py’

In [8]:
import datetime
#from emissions_eq_pyc import *
from emissions_eq import *
from google.colab import files
from tracelib import *

In [9]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]
traces = pd.read_csv(filename)

Saving deita.csv to deita (6).csv


In [11]:
traces['nox'] = traces.apply(lambda row: row.noxme + row.noxae, axis=1)
traces['sox'] = traces.apply(lambda row: row.soxme + row.soxae, axis=1)
traces['co2'] = traces.apply(lambda row: row.co2me + row.co2ae, axis=1)
traces['co'] = traces.apply(lambda row: row.come + row.coae, axis=1)

Cast ts a datetime (tipo de datos consciente del tiempo)

In [12]:
traces['ts'] = pd.to_datetime(traces['ts'])

# Acumulado de cada traza

In [13]:
traces.drop(columns = ['uid', 'gid', 'lat', 'lon']).groupby('sid').sum()

,noxme,noxae,soxme,soxae,co2me,co2ae,come,coae,nox,sox,co2,co
sid,,,,,,,,,,,,
1,0.049752,74.010505,0.001531,2.277631,2.386915,3550.756481,4.417091e-02,0,74.060257,2.279162,3553.143397,4.417091e-02
2,1599.829540,419.392863,49.233835,12.906575,76754.037765,20120.953394,9.267876e+07,0,2019.222403,62.140410,96874.991159,9.267876e+07
5,1.485912,246.701684,0.045728,7.592103,71.288704,11835.854938,1.907951e+01,0,248.187596,7.637831,11907.143642,1.907951e+01


# Traza 1

In [14]:
trace1 = traces[traces.sid == 1]

In [15]:
pollutant = "co2" # nox, sox, co2, co
plot_trace(trace1, height = 600, pollutant = pollutant)
plot_trace_line(trace1, height = 600)

/content/tracelib.py:20: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.



/content/tracelib.py:10: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.



# Speed + Acceleration

In [16]:
speed = calculateSpeedAccPandas(trace1)

In [17]:
plot_line(speed, 'id', 'speed', title="Speed")
plot_line(speed, 'id', 'acc', title="Acceleration")
plot_line(speed, 'id', 'timeDiff', title="Time between messages")

# Emissions

In [18]:
ship = getShipProfile()

In [19]:
ship['type'] # Try changing this value to "Yatch" and take a look to the auxiliary engine!

'Passenger/Ro-Ro Cargo Ship'

In [20]:
emis = computeEmissions(ship, speed['speed'], trace1['ts'])

In [24]:
emis.drop(columns = ['id']).sum()


speed        5.974150
noxme        0.047916
soxme        0.001475
co2me        2.298858
come         0.034343
noxae       74.010505
soxae        2.277631
co2ae     3550.756481
coae         0.000000
pme         13.285217
pae      20520.000000
dtype: float64

In [25]:
plot_line(emis, 'id', 'soxme', title='SOx ME')
plot_line(emis, 'id', 'soxae', title='SOx AE')
plot_line(emis, 'id', 'noxme', title='NOx ME')
plot_line(emis, 'id', 'noxae', title='NOx AE')
plot_line(emis, 'id', 'co2me', title='CO2 ME')
plot_line(emis, 'id', 'co2ae', title='CO2 AE')
plot_line(emis, 'id', 'come', title='CO ME')


In [26]:
emis

,id,speed,noxme,soxme,co2me,come,noxae,soxae,co2ae,coae,pme,pae
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,12.335084,0.379605,591.792747,0,0.000000,3420
1,1,0.888048,0.003464,0.000107,0.166199,0.005675,12.335084,0.379605,591.792747,0,0.960471,3420
2,2,1.025747,0.005338,0.000164,0.256118,0.001356,12.335084,0.379605,591.792747,0,1.480116,3420
3,3,1.217476,0.008926,0.000275,0.428253,0.004510,12.335084,0.379605,591.792747,0,2.474893,3420
4,4,1.626578,0.021287,0.000655,1.021277,0.016065,12.335084,0.379605,591.792747,0,5.902010,3420
5,5,1.216300,0.008900,0.000274,0.427012,0.006736,12.335084,0.379605,591.792747,0,2.467726,3420
